# Python for R users
# Part 5: Data wrangling and analysis using Pandas

In this notebook we will explore how to use the Pandas library to work with data.  Pandas has a structure that is conceptually similar to R in some ways, but very different in others. There are some great tutorials on Pandas linked [here](http://www.data-analysis-in-python.org/3_pandas.html).

First we need to tell Jupyter to let us use R within this Python notebook, and import some necessary libraries.

In [2]:
import pandas
import numpy
from pprint import pprint

%load_ext rpy2.ipython

## Data types in Pandas

There are two main types of data structures in Pandas that we will need to use: Series, and Data Frames.  Let's first introduce Series.

### Series in Pandas

A Series is a one-dimensional data structure, akin to a vector.  The main difference between a Pandas Series and a list or vector is that the Series contains some additional indexing information.  To see how this works, let's generate some numbers and put them in a Series.

In [89]:
numpy.random.seed(12345)  # fix the random seed for reproducibility

s = pandas.Series(numpy.random.randn(5))
print(s)

0   -0.204708
1    0.478943
2   -0.519439
3   -0.555730
4    1.965781
dtype: float64


The index for this Series by default is a series of integers starting at zero.  However, we can also specify an index explicity. Let's say that our 5 numbers come from 5 different people, and we want to use their names as the index:

In [90]:
s_indexed = pandas.Series(numpy.random.randn(5),
                  index=['Lisa', 'Sue', 'Karen', 'Lucy', 'Helen'])
print(s_indexed)

Lisa     1.393406
Sue      0.092908
Karen    0.281746
Lucy     0.769023
Helen    1.246435
dtype: float64


We can also access the index directly using the ```.index``` element of the Series:

In [91]:
s_indexed.index

Index(['Lisa', 'Sue', 'Karen', 'Lucy', 'Helen'], dtype='object')

And we can set the index using that element as well:

In [92]:
s.index = ['Lisa', 'Sue', 'Karen', 'Lucy', 'Helen']
print(s)

Lisa    -0.204708
Sue      0.478943
Karen   -0.519439
Lucy    -0.555730
Helen    1.965781
dtype: float64


A nice feature is that we can then access the data using the index:

In [93]:
# access a single element
print(s['Helen'])

# access a range of elements
print(s['Karen':'Helen'])


1.9657805725027142
Karen   -0.519439
Lucy    -0.555730
Helen    1.965781
dtype: float64


We can also index based on the values of the series --- for example, we can find all entries with values less than zero:

In [94]:
print(s[s<0])

Lisa    -0.204708
Karen   -0.519439
Lucy    -0.555730
dtype: float64


This works because ```s < 0``` returns a Boolean series, and using a Boolean series as a index to another Series will return all of the places where the Boolean series is true:

In [95]:
print(s < 0)

Lisa      True
Sue      False
Karen     True
Lucy      True
Helen    False
dtype: bool


Sometimes you may want to extract the values from a Series back into a Numpy array. You can do this using the ```.values``` element of the Series

In [96]:
s.values

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

You can sort the Series according to the data values using the ```.sort_values()``` function:

In [97]:
s_sorted = s.sort_values()
print(s_sorted)

Lucy    -0.555730
Karen   -0.519439
Lisa    -0.204708
Sue      0.478943
Helen    1.965781
dtype: float64


Now let's say that you wanted to only take the top three of the sorted list. 

In [98]:
s_sorted[-3:]

Lisa    -0.204708
Sue      0.478943
Helen    1.965781
dtype: float64

## Data frames

The concept of a data frame will be very familiar to R users, and the Pandas library provides a very similar functionality for Python.  We will spend a good bit of time on data frames because of their importance.  We will move to working with real data shortly, but we will start with a simple example.  First let's create a data frame in R with two variables.

In [99]:
%%R

x <- c(1, 2, 3, 4, 5)
y <- x * 2 + 3

df <- data.frame(x = x, y = y)
print(df)

# access one of the variables in the data frame
print(df['y'])

# summarize the variables in the data frame
summary(df)

  x  y
1 1  5
2 2  7
3 3  9
4 4 11
5 5 13
   y
1  5
2  7
3  9
4 11
5 13
       x           y     
 Min.   :1   Min.   : 5  
 1st Qu.:2   1st Qu.: 7  
 Median :3   Median : 9  
 Mean   :3   Mean   : 9  
 3rd Qu.:4   3rd Qu.:11  
 Max.   :5   Max.   :13  


Now let's create an analogous data frame in Python.  There are various ways to get data into a data frame - in this case we will specify them as a dictionary to the ```pandas.DataFrame()``` function.

In [100]:
x = numpy.array([1, 2, 3, 4, 5])
y = x * 2 + 3

df = pandas.DataFrame({'x': x, 'y': y})
print(df)

# access one of the variables in the data frame
# note that this becomes a pandas Series
print(df.y)

# summarize the variables in the data frame
df.describe()

   x   y
0  1   5
1  2   7
2  3   9
3  4  11
4  5  13
0     5
1     7
2     9
3    11
4    13
Name: y, dtype: int64


,x,y
count,5.000000,5.000000
mean,3.000000,9.000000
std,1.581139,3.162278
min,1.000000,5.000000
25%,2.000000,7.000000
50%,3.000000,9.000000
75%,4.000000,11.000000
max,5.000000,13.000000


### Indexing columns

There are several different ways to access specific elements of a Pandas data frame.  First, let's look at accessing the columns. There are two ways to access a column by name. You have already seen one above, using the dot notation.  A column can also be accessed by putting its name in brackets, as in R:

In [101]:
print(df['y'])

0     5
1     7
2     9
3    11
4    13
Name: y, dtype: int64


The columns can also be accessed based on their numeric position, using the standard indexing that saw for Numpy arrays.  This is done using the ```.iloc``` operator to the data frame.  The first dimension in the iloc argument refers to the row, and the second to the column.  Thus, if we wanted to access the first three elements in the second columns, we would use:

In [102]:
df.iloc[:3, 1]  # columns are indexed from zero, so 1 refers to the second column

0    5
1    7
2    9
Name: y, dtype: int64

### Indexing rows

There are two different ways to access a row in a Pandas data frame. 

The first you have already seen in the previous section, using the ```.iloc``` operator to index them numerically based on their position in the array.  

The second relies upon the data frame's index, using the ```.loc``` operator.  Let's say that we want to extract the third row, which in this case has an index value of 2.  We could do that as follows:

In [103]:
print(df.loc[2])

x    3
y    9
Name: 2, dtype: int64


Often we would like a more descriptive index.  For example, let's say that our five rows refer to five different cities in California.  We can set the index using the ```index``` operator:

In [104]:
df.index = ['Los Angeles', 'Santa Barbara', 'Sacramento', 'Bakersfield', 'San Francisco']
print(df)

               x   y
Los Angeles    1   5
Santa Barbara  2   7
Sacramento     3   9
Bakersfield    4  11
San Francisco  5  13


Now, if we want to access the data for Sacremento, we can do that using the ```.loc``` operator:


In [105]:
df.loc['Sacramento']

x    3
y    9
Name: Sacramento, dtype: int64

## Reading and working with data

Pandas is the tool that one would most often use to load and work with data sets.  As an example, let's read a real dataset from disk and show how we would work with it using Pandas.

We will use a real dataset from [Eisenberg et al, 2019](nature.com/articles/s41467-019-10301-1), which contains data for 522 individuals on 192 variables derived from a set of psychological tasks.  Let's say that we want to know whether the stop signal reaction time (SSRT - measured across several different tasks) is related to self-reported impulsivity (as measured using a number of different surveys).  

First we load the data.  The subject codes are contained in the first column, and we tell pandas to use those codes as the index for the data frame.

In [106]:
data = pandas.read_csv('meaningful_variables_clean.csv', index_col=0)
data.head()

,adaptive_n_back.hddm_drift,adaptive_n_back.hddm_drift_load,adaptive_n_back.hddm_non_decision,adaptive_n_back.hddm_thresh,adaptive_n_back.mean_load.logTr,angling_risk_task_always_sunny.keep_adjusted_clicks,angling_risk_task_always_sunny.keep_coef_of_variation,angling_risk_task_always_sunny.release_adjusted_clicks,angling_risk_task_always_sunny.release_coef_of_variation.logTr,attention_network_task.alerting_hddm_drift,...,two_stage_decision.model_based,two_stage_decision.model_free,two_stage_decision.perseverance,upps_impulsivity_survey.lack_of_perseverance,upps_impulsivity_survey.lack_of_premeditation,upps_impulsivity_survey.negative_urgency,upps_impulsivity_survey.positive_urgency,upps_impulsivity_survey.sensation_seeking,writing_task.neutral_probability,writing_task.positive_probability
s001,NaN,NaN,NaN,NaN,NaN,24.250000,8.051643,17.090909,2.199145,-0.558765,...,NaN,NaN,NaN,1.5,1.909091,1.583333,2.142857,3.416667,0.201794,0.189649
s002,1.187554,-0.410072,0.090573,1.863749,0.182322,20.260870,17.184077,13.000000,1.978096,-0.189757,...,-0.393831,-0.122940,-0.895623,1.5,2.090909,3.000000,3.285714,3.083333,0.817275,0.192216
s003,2.215680,-0.726543,0.025634,2.150399,0.371564,13.080000,5.992495,10.086957,1.833585,-0.430268,...,NaN,NaN,NaN,2.4,2.363636,3.000000,2.785714,2.833333,0.412338,0.231957
s004,2.065906,-0.507549,0.037627,2.385402,0.854415,13.464286,2.937398,8.928571,1.181606,-0.485213,...,-0.417454,-0.083370,-0.591512,1.5,1.636364,1.583333,1.142857,1.916667,0.888122,0.256883
s005,3.221946,-1.235354,0.285800,1.580276,0.500775,36.363636,13.145868,30.500000,2.453175,-0.238605,...,0.665538,0.103557,1.355168,1.7,1.363636,1.750000,1.000000,2.833333,0.886852,0.405901


First we need to select the variables that we will use for the analysis. First, let's find the SSRT variables.  To do this, we will loop through all of the variable names and find the ones that include "ssrt".  In R we would access the column names using ```names()``` whereas in Python we access them using the ```.columns``` operator.

In [107]:
data.columns

Index(['adaptive_n_back.hddm_drift', 'adaptive_n_back.hddm_drift_load',
       'adaptive_n_back.hddm_non_decision', 'adaptive_n_back.hddm_thresh',
       'adaptive_n_back.mean_load.logTr',
       'angling_risk_task_always_sunny.keep_adjusted_clicks',
       'angling_risk_task_always_sunny.keep_coef_of_variation',
       'angling_risk_task_always_sunny.release_adjusted_clicks',
       'angling_risk_task_always_sunny.release_coef_of_variation.logTr',
       'attention_network_task.alerting_hddm_drift',
       ...
       'two_stage_decision.model_based', 'two_stage_decision.model_free',
       'two_stage_decision.perseverance',
       'upps_impulsivity_survey.lack_of_perseverance',
       'upps_impulsivity_survey.lack_of_premeditation',
       'upps_impulsivity_survey.negative_urgency',
       'upps_impulsivity_survey.positive_urgency',
       'upps_impulsivity_survey.sensation_seeking',
       'writing_task.neutral_probability',
       'writing_task.positive_probability'],
      dtype='o

Let's loop through all of the column names and save the ones that have 'ssrt' in their name.  We will learn more about processing strings in a later section; here we will introduce the ```.find()``` operator that is present for strings.  This operator tells us the location of a particular string within another string, or returns -1 if the string is not present.  For example:

In [108]:
a = 'This is a string'
print(a.find('is'))
print(a.find('number'))

2
-1


Here is how we would do it for the variables.

In [109]:
# first find SSRT variables

ssrt_variables = []  # empty list to save names
for c in data.columns:
    if c.find('.SSRT') > -1:
        ssrt_variables.append(c)
        
print(ssrt_variables)

# find BIS-11

bis11_variables = []  # empty list to save names
for c in data.columns:
    if c.find('bis11') > -1:
        bis11_variables.append(c)
        
print(bis11_variables)


['motor_selective_stop_signal.SSRT', 'stim_selective_stop_signal.SSRT', 'stop_signal.SSRT_high.logTr', 'stop_signal.SSRT_low']
['bis11_survey.Attentional', 'bis11_survey.Motor.logTr', 'bis11_survey.Nonplanning']


So far we have found the variables of interest for SSRT and BIS-11.  However, you can see above that we have repeated the same code in two places, which is bad form.  What we should do instead is find a way to loop through the variables we are interested in, so that if we decide that we want to add more then we can do that easily later.  First, we should set up a dictionary that contains all of the search strings for each data type.

In [110]:
search_strings = {
    'SSRT': '.SSRT',
    'BIS-11': 'bis11',
    'UPPS-P': 'upps',
    'Dickman': 'dickman'
}
print(search_strings)

{'SSRT': '.SSRT', 'BIS-11': 'bis11', 'UPPS-P': 'upps', 'Dickman': 'dickman'}


Now let's adapt the code above to loop through the different variables.  Tip: Putting a dictionary as the sequence in a for loop causes it to loop over all of the keys in the dictionary.

In [111]:
variable_names = {}
for ss in search_strings:
    variable_names[ss] = []  # create empty list to store matching names for this string
    for c in data.columns:
        if c.find(search_strings[ss]) > -1:
            variable_names[ss].append(c)

pprint(variable_names)

{'BIS-11': ['bis11_survey.Attentional',
            'bis11_survey.Motor.logTr',
            'bis11_survey.Nonplanning'],
 'Dickman': ['dickman_survey.functional'],
 'SSRT': ['motor_selective_stop_signal.SSRT',
          'stim_selective_stop_signal.SSRT',
          'stop_signal.SSRT_high.logTr',
          'stop_signal.SSRT_low'],
 'UPPS-P': ['upps_impulsivity_survey.lack_of_perseverance',
            'upps_impulsivity_survey.lack_of_premeditation',
            'upps_impulsivity_survey.negative_urgency',
            'upps_impulsivity_survey.positive_urgency',
            'upps_impulsivity_survey.sensation_seeking']}


Now that we have found all of the relevant variables, let's create a new data frame that only includes those variables.  First we will create a list with all of the matching variables, and then filter the data frame for these columns.

In [112]:
variables_to_keep = []
for v in variable_names:
    variables_to_keep += variable_names[v]

data_selected = data[variables_to_keep]
data_selected.columns

Index(['motor_selective_stop_signal.SSRT', 'stim_selective_stop_signal.SSRT',
       'stop_signal.SSRT_high.logTr', 'stop_signal.SSRT_low',
       'bis11_survey.Attentional', 'bis11_survey.Motor.logTr',
       'bis11_survey.Nonplanning',
       'upps_impulsivity_survey.lack_of_perseverance',
       'upps_impulsivity_survey.lack_of_premeditation',
       'upps_impulsivity_survey.negative_urgency',
       'upps_impulsivity_survey.positive_urgency',
       'upps_impulsivity_survey.sensation_seeking',
       'dickman_survey.functional'],
      dtype='object')

Now let's clean up the data by removing all of the missing data, using the ```.dropna()``` operator.  

In [113]:
data_selected = data_selected.dropna()
print(data_selected.shape)

(320, 13)


Now we need to compute the average of all SSRT variables and the average of all of the impulsivity variables.  

In [114]:
data_selected['mean_SSRT'] = data_selected[variable_names['SSRT']].mean(axis=1)

# create a list containing all of the variables except those for SSRT
# using set operations

impulsivity_variables = set(variables_to_keep).difference(variable_names['SSRT'])

# we need to turn the set back into a list so it can be used as an index
data_selected['mean_impulsivity'] = data_selected[list(impulsivity_variables)].mean(axis=1)


Now we can compute the correlation between those two mean variables of interest, using the built-in correlation method ```.corr()```.

In [115]:
data_selected[['mean_impulsivity', 'mean_SSRT']].corr()

,mean_impulsivity,mean_SSRT
mean_impulsivity,1.000000,-0.043986
mean_SSRT,-0.043986,1.000000


## Joining datasets

Often we will want to combine data that are stored in multiple files.  Just like R, pandas provides tools for merging datasets using a common set of indices, using the ```.merge()``` operator.  Let's say that we want to combine our data above with some demographic data, so that we can ask whether impulsivity and SSRT are related to ever having been arrested in one's life.  These data are stored in a separate file (demographics.csv).  First we load the file:

In [116]:
demogdata = pandas.read_csv('demographics.csv', index_col=0)
demogdata.head()


,Sex,Age,Race,OtherRace,HispanicLatino,HighestEducation,HeightInches,WeightPounds,RelationshipStatus,DivorceCount,...,CoffeeCupsPerDay,TeaCupsPerDay,CaffienatedSodaCansPerDay,CaffieneOtherSourcesDayMG,GamblingProblem,TrafficTicketsLastYearCount,TrafficAccidentsLifeCount,ArrestedChargedLifeCount,MotivationForParticipation,MotivationOther
s001,1,27,White,NaN,1,3,62,110,2,0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,money,NaN
s002,0,35,White,NaN,0,2,72,240,2,0,...,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,money,NaN
s003,0,25,White,NaN,0,4,73,185,1,0,...,1.0,0.0,0.0,90.0,0.0,0.0,1.0,0.0,money,NaN
s004,0,35,White,NaN,0,4,71,190,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,money,NaN
s005,0,36,Black or African American,NaN,0,3,76,175,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,3.0,5.0,money,NaN


There is a variable called "ArrestedChargedLifeCount" that contains the self-reported number of times that a person has been arrested in their life.  Let's create a new variable that is True if the person has ever been arrested:

In [117]:
demogdata['EverArrested'] = demogdata.ArrestedChargedLifeCount > 0

Note: When you are referring to the value of a variable within a data frame, you can use either ```dataframe.VariableName``` or ```dataframe['VariableName']```.  However, when you are adding a new variable to a data frame, you must use the latter syntax.

Now let's join the new variable with the data from above.  We want to only include cases that are present in both datasets, so we use what is called an "inner join" (see [the pandas.DataFrame.join help](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) for more on this).  

In [118]:
data_selected_with_arrest = data_selected[['mean_impulsivity', 'mean_SSRT']].join(demogdata['EverArrested'], how = 'inner')
data_selected_with_arrest.head()

,mean_impulsivity,mean_SSRT,EverArrested
s004,1.911021,200.211691,False
s005,2.088186,271.213365,True
s009,2.961991,159.057108,False
s011,2.060847,176.495160,False
s012,2.275128,237.863131,False


## Grouping variables using groupby()



Now we want to summarize the data by the different groups: Arrested vs. never arrested. Here we can use a method called ```.groupby()``` which is similar to the ```group_by()``` function in dplyr.  First let's group the data and compute the mean for each group on each variable, and then compute normal confidence intervals for each mean.

In [119]:
# compute means for each group/variable
mean_by_group = data_selected_with_arrest.groupby('EverArrested').mean()
print(mean_by_group)

# comnpute standard errors
stderr_by_group = data_selected_with_arrest.groupby('EverArrested').std()/numpy.sqrt(data_selected_with_arrest.shape[0])
print(stderr_by_group)

# compute normal confidence intervals
upper_ci = mean_by_group + stderr_by_group*1.96
upper_ci.columns = [i + '_upperCI' for i in upper_ci.columns] # fix names

lower_ci = mean_by_group - stderr_by_group*1.96
lower_ci.columns = [i + '_lowerCI' for i in lower_ci.columns] # fix names

# add back into a single data frame
results = pandas.concat([mean_by_group, upper_ci, lower_ci], axis=1) #, ))
# reorder the columns
result_columns = results.columns.tolist()
result_columns.sort()
results = results[result_columns]
results


              mean_impulsivity   mean_SSRT
EverArrested                              
False                 2.179763  230.255239
True                  2.351907  226.701396
              mean_impulsivity  mean_SSRT
EverArrested                             
False                 0.021448   1.989571
True                  0.023668   2.073708


,mean_SSRT,mean_SSRT_lowerCI,mean_SSRT_upperCI,mean_impulsivity,mean_impulsivity_lowerCI,mean_impulsivity_upperCI
EverArrested,,,,,,
False,230.255239,226.355680,234.154798,2.179763,2.137725,2.221802
True,226.701396,222.636929,230.765863,2.351907,2.305518,2.398296


Eyeballing these results, it appears that there is a difference in impulsivity between the groups.  We will return to this later when we move on to statistical inference.  Before we leave, let's save the data to a csv file so that we can read it back in when we return to this later.  This is easy using the ```.to_csv()``` method with the data frame.

In [120]:
data_selected_with_arrest.to_csv('arrest_ssrt_impulsivity.csv')

## Moving between wide and long data formats

It's common in data analysis to need to move between wide and long data formats.  Here we will work with item-level responses from a set of surveys obtained from the UH2 project, which are stored in subjects_x_items.csv. The items are stored in a wide format, with the item numbers appended to the name of the survey (e.g. eating_survey.02, ... eating_survey.19).  Let's load the data from the eating survey and move the data from wide to long format, first in R.

In [18]:
%%R

library(readr)
library(dplyr)
library(tidyr)

eating_data = read_csv('subject_x_items.csv') %>%
    select(c(starts_with("eating_survey"), 'worker')) %>% # select only eating survey items 
    tibble::rownames_to_column('subjectID')

# convert to long format
eating_data_long <- gather(eating_data, item, response, -subjectID, factor_key=TRUE)
head(eating_data_long)


# A tibble: 6 x 3
  subjectID item             response
  <chr>     <fct>            <chr>   
1 1         eating_survey.02 3       
2 2         eating_survey.02 3       
3 3         eating_survey.02 1       
4 4         eating_survey.02 2       
5 5         eating_survey.02 2       
6 6         eating_survey.02 1       


In [26]:
eating_data = pandas.read_csv('subject_x_items.csv', index_col=0)
keep_cols = [i for i in eating_data.columns if i.find('eating_survey')==0]
eating_data = eating_data[keep_cols]
eating_data['subjectID'] = eating_data.index

eating_data_long = pandas.melt(eating_data,
                               id_vars=['subjectID'],
                               var_name='item',
                               value_name='response')
eating_data_long.head()

,subjectID,item,response
0,s001,eating_survey.02,3.0
1,s002,eating_survey.02,3.0
2,s003,eating_survey.02,1.0
3,s004,eating_survey.02,2.0
4,s005,eating_survey.02,2.0
